In [25]:
import boto3
import pandas as pd
import numpy as np
from io import StringIO
import json
from io import BytesIO
import ijson
from pandas import json_normalize

# the below extension properly formats a cell after it is run
%load_ext nb_black 

# Set the maximum number of rows to 200
pd.set_option("display.max_rows", 200)
import pandas as pd

# Set the maximum number of columns to 200
pd.set_option('display.max_columns', 200)


The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

Loading the CSV's from AWS

In [2]:
# Creating an S3 client object
s3 = boto3.client('s3')

# Specifying the name of the bucket
bucket_name = 'capstonehaystacks'

# List of CSV files to download
csv_files = [
    'atlanta_cbsa_zip.csv',
    'core_geo_dataset.csv',
    'crime_rating_zipcode.csv',
    'elementary_schools.csv',
    'GA_LISTINGS_SALES_V2.csv',
    'GA_LISTINGS_SALES.csv',
    'high_schools.csv',
    'middle_schools.csv'
]

# Dictionary to store the dataframes
dataframes = {}

# Downloading the files from S3 and reading them into pandas dataframes
for file_name in csv_files:
    s3.download_file(bucket_name, file_name, file_name)
    dataframes[file_name[:-4]] = pd.read_csv(file_name, index_col=False)

# Access the dataframes using their respective keys
atlanta_cbsa_zip_df = dataframes['atlanta_cbsa_zip']
core_geo_dataset_df = dataframes['core_geo_dataset']
crime_rating_zipcode_df = dataframes['crime_rating_zipcode']
elementary_schools_df = dataframes['elementary_schools']
GA_LISTINGS_SALES_V2_df = dataframes['GA_LISTINGS_SALES_V2']
GA_LISTINGS_SALES_df = dataframes['GA_LISTINGS_SALES']
high_schools_df = dataframes['high_schools']
middle_schools_df = dataframes['middle_schools']



<IPython.core.display.Javascript object>

In [3]:
# Looping through the keys in the dataframes dictionary and drop the 'Unnamed: 0' column
for key in dataframes.keys():
    dataframes[key].drop(columns=['Unnamed: 0'], inplace=True)


<IPython.core.display.Javascript object>

In [4]:
print("Columns in atlanta_cbsa_zip_df:")
print(atlanta_cbsa_zip_df.columns)

print("\nColumns in core_geo_dataset_df:")
print(core_geo_dataset_df.columns)

print("\nColumns in crime_rating_zipcode_df:")
print(crime_rating_zipcode_df.columns)

print("\nColumns in elementary_schools_df:")
print(elementary_schools_df.columns)

print("\nColumns in GA_LISTINGS_SALES_V2_df:")
print(GA_LISTINGS_SALES_V2_df.columns)

print("\nColumns in GA_LISTINGS_SALES_df:")
print(GA_LISTINGS_SALES_df.columns)

print("\nColumns in high_schools_df:")
print(high_schools_df.columns)

print("\nColumns in middle_schools_df:")
print(middle_schools_df.columns)


Columns in atlanta_cbsa_zip_df:
Index(['census_cbsa_geoid', 'census_cbsa_name', 'census_cbsa_lsad',
       'census_cbsa_lat', 'census_cbsa_lon', 'census_zcta5_geoid',
       'census_zcta5_lat', 'census_zcta5_lon'],
      dtype='object')

Columns in core_geo_dataset_df:
Index(['census_zcta5_geoid', 'census_zcta5_lat', 'census_zcta5_lon',
       'census_tract_geoid', 'census_tract_lat', 'census_tract_lon'],
      dtype='object')

Columns in crime_rating_zipcode_df:
Index(['census_state_abbr', 'census_zcta5_geoid', 'census_cbsa_geoid_count',
       'census_cbsa_geoid_list', 'overall_crime_grade', 'violent_crime_grade',
       'violent_crime_total_rate', 'violent_crime_assault_rate',
       'violent_crime_robbery_rate', 'violent_crime_rape_rate',
       'violent_crime_murder_rate', 'property_crime_grade',
       'property_crime_total_rate', 'property_crime_theft_rate',
       'property_crime_vehicle_theft_rate', 'property_crime_burglary_rate',
       'property_crime_arson_rate', 'other_cri

<IPython.core.display.Javascript object>

Index_col=False didnt work so Im going to drop the first column for each data frame

Looking into the GA_listings_sales files to find the difference between them

In [6]:
GA_LISTINGS_SALES_df.shape

(31064, 20)

In [7]:
GA_LISTINGS_SALES_V2_df.shape

(31064, 21)

In [8]:
GA_LISTINGS_SALES_df.columns

Index(['latitude', 'longitude', 'city', 'county_name', 'beds', 'baths_full',
       'baths_half', 'square_footage', 'lot_size', 'year_built', 'details',
       'special_features', 'unit_count', 'price', 'transaction_type',
       'listing_status', 'listing_special_features', 'census_state_name',
       'census_county_name', 'zip'],
      dtype='object')

In [9]:
GA_LISTINGS_SALES_V2_df.columns

Index(['latitude', 'longitude', 'full_street_address', 'city', 'county_name',
       'beds', 'baths_full', 'baths_half', 'square_footage', 'lot_size',
       'year_built', 'details', 'special_features', 'unit_count', 'price',
       'transaction_type', 'listing_status', 'listing_special_features',
       'census_state_name', 'census_county_name', 'zip'],
      dtype='object')

It looks like the difference is V2 has a full_street_address column.  Going forward I will work with V2

Next I'm going to look into the two json files to figure out how to properly convert them into a dataframe

In [73]:
# This is a large JSON file and the below allows me to view just a portion of it so I can
# determine how to flatten it

# COMMENTED OUT SINCE IT EXPOSES AN API KEY


# # Setting the bucket and file names
# bucket_name = 'capstonehaystacks'
# file_name = 'all_zips_grocery_store.json'

# # Reading the file from the S3 bucket
# response = s3.get_object(Bucket=bucket_name, Key=file_name)

# # Wrapping the response's body in a BytesIO object so ijson can read it
# file = BytesIO(response['Body'].read())

# # Using 'item' as the prefix for each JSON object.
# items = ijson.items(file, 'item')

# # Iterating through the JSON objects
# for index, item in enumerate(items):
#     # Print the current item
#     print(f"Item {index}: {item}")

#     # Breaking the loop here at 10
#     if index >= 10:
#         break


<IPython.core.display.Javascript object>

I can now flatten the data after seeing how it is nested. I'm doing this locally instead of using the version in AWS. 

In [26]:
# Loading the json file from local disk
with open("all_zips_grocery_store.json", "r") as file:
    data = json.load(file)

# Flattening the data
flattened_data = json_normalize(data, record_path=["responce", "results"], meta=[["index"], ["census_zcta_geoid"], ["params", "key"], ["params", "location"], ["params", "radius"], ["params", "type"], ["responce", "status"]], errors="ignore")

# Converting to a dataframe
grocery_df = pd.DataFrame(flattened_data)


<IPython.core.display.Javascript object>

In [30]:
print(grocery_df.shape)
print(grocery_df.dtypes)

(402803, 31)
icon                                object
icon_background_color               object
icon_mask_base_uri                  object
name                                object
place_id                            object
reference                           object
scope                               object
types                               object
vicinity                            object
geometry.location.lat              float64
geometry.location.lng              float64
geometry.viewport.northeast.lat    float64
geometry.viewport.northeast.lng    float64
geometry.viewport.southwest.lat    float64
geometry.viewport.southwest.lng    float64
business_status                     object
plus_code.compound_code             object
plus_code.global_code               object
permanently_closed                  object
photos                              object
rating                             float64
user_ratings_total                 float64
opening_hours.open_now              objec

<IPython.core.display.Javascript object>

It is a large file and I'm now going to remove all rows outside of atlanta

In [33]:
# Converting the "census_zcta_geoid" column to integers
grocery_df["census_zcta_geoid"] = grocery_df["census_zcta_geoid"].astype(int)

<IPython.core.display.Javascript object>

In [32]:
# Using the zip codes in this file to filter the grocery_df
zip_list = atlanta_cbsa_zip_df["census_zcta5_geoid"].tolist()

<IPython.core.display.Javascript object>

In [35]:
# Creating a boolean mask
mask = grocery_df["census_zcta_geoid"].isin(zip_list)

# Filtering the dataframe based on the boolean mask
grocery_atlanta_df = grocery_df[mask]

<IPython.core.display.Javascript object>

In [46]:
# Dropping columns that arent useful
grocery_atlanta_df = grocery_atlanta_df.drop(['icon', 'icon_background_color', 'params.location', 'params.key', 'params.radius', 'params.type', 'icon_mask_base_uri', 'reference', 'scope', 'index', 'responce.status'], axis=1).reset_index(drop=True)

<IPython.core.display.Javascript object>

I'm now doing the same process with all_zips_restaurant

In [50]:
# Loading the json file from local disk
with open("all_zips_restaurant.json", "r") as file:
    data = json.load(file)

# Flattening the data
flattened_data = json_normalize(data, record_path=["responce", "results"], meta=[["index"], ["census_zcta_geoid"], ["params", "key"], ["params", "location"], ["params", "radius"], ["params", "type"], ["responce", "status"]], errors="ignore")

# Converting to a dataframe
restaurant_df = pd.DataFrame(flattened_data)


<IPython.core.display.Javascript object>

In [51]:
print(restaurant_df.shape)
print(restaurant_df.dtypes)

(198343, 31)
business_status                     object
icon                                object
icon_background_color               object
icon_mask_base_uri                  object
name                                object
photos                              object
place_id                            object
price_level                        float64
rating                             float64
reference                           object
scope                               object
types                               object
user_ratings_total                 float64
vicinity                            object
geometry.location.lat              float64
geometry.location.lng              float64
geometry.viewport.northeast.lat    float64
geometry.viewport.northeast.lng    float64
geometry.viewport.southwest.lat    float64
geometry.viewport.southwest.lng    float64
opening_hours.open_now              object
plus_code.compound_code             object
plus_code.global_code               objec

<IPython.core.display.Javascript object>

In [52]:
# Converting the "census_zcta_geoid" column to integers
restaurant_df["census_zcta_geoid"] = restaurant_df["census_zcta_geoid"].astype(int)

<IPython.core.display.Javascript object>

In [55]:
# Creating a boolean mask with the zip code list created earlier
mask = restaurant_df["census_zcta_geoid"].isin(zip_list)

# Filter the dataframe based on the boolean mask
restaurant_atlanta_df = restaurant_df[mask]

<IPython.core.display.Javascript object>

In [57]:
# Dropping columns that arent useful and resetting index
restaurant_atlanta_df = restaurant_atlanta_df.drop(['icon', 'icon_background_color', 'params.location', 'params.key', 'params.radius', 'params.type', 'icon_mask_base_uri', 'reference', 'scope', 'index', 'responce.status'], axis=1).reset_index(drop=True)

<IPython.core.display.Javascript object>

Combining the two files, dropping duplicates and saving in AWS and locally

In [64]:
# Combining the two dataframes by concating vertically 
poi_df = pd.concat([restaurant_atlanta_df, grocery_atlanta_df], ignore_index=True)

<IPython.core.display.Javascript object>

In [70]:
# Checking or duplicates
poi_df.duplicated(subset=['place_id']).sum()

1075

<IPython.core.display.Javascript object>

In [71]:
#Dropping duplicates
poi_df.drop_duplicates(subset='place_id', inplace=True)

<IPython.core.display.Javascript object>

In [75]:
poi_df.shape

(5213, 20)

<IPython.core.display.Javascript object>

In [78]:
# Saving locally
poi_df.to_csv("poi_df.csv", index=False)

<IPython.core.display.Javascript object>

In [79]:
# Saving to AWS
file_name = "poi_df.csv"

# Upload the file to S3
s3.upload_file(file_name, bucket_name, file_name)


<IPython.core.display.Javascript object>

### Processing GA_LISTINGS_SALES_V2_df and removing data that represents property outside of Atlanta

In [28]:
# Zip column is currently a string value
GA_LISTINGS_SALES_V2_df.dtypes

latitude                    float64
longitude                   float64
full_street_address          object
city                         object
county_name                  object
beds                        float64
baths_full                  float64
baths_half                  float64
square_footage              float64
lot_size                    float64
year_built                  float64
details                      object
special_features              int64
unit_count                  float64
price                         int64
transaction_type              int64
listing_status                int64
listing_special_features      int64
census_state_name            object
census_county_name           object
zip                          object
dtype: object

In [84]:
# Converting the zip column to a numeric type and dropping any non-numeric or missing values
GA_LISTINGS_SALES_V2_df['zip'] = pd.to_numeric(GA_LISTINGS_SALES_V2_df['zip'], errors='coerce')
GA_LISTINGS_SALES_V2_df = GA_LISTINGS_SALES_V2_df.dropna(subset=['zip'])

# Converting the zip column to an integer
GA_LISTINGS_SALES_V2_df.loc['zip'] = GA_LISTINGS_SALES_V2_df['zip'].astype(int)

# Filtering GA_LISTINGS_SALES_V2_df to keep only the rows with Atlanta zip codes
GA_LISTINGS_SALES_V2_df = GA_LISTINGS_SALES_V2_df[GA_LISTINGS_SALES_V2_df['zip'].isin(zip_list)]


<IPython.core.display.Javascript object>

In [86]:
GA_LISTINGS_SALES_V2_df.isna().sum()


latitude                        0
longitude                       0
full_street_address             0
city                            0
county_name                     0
beds                         7764
baths_full                   7793
baths_half                  12875
square_footage               8690
lot_size                      520
year_built                   6278
details                         0
special_features                0
unit_count                  16971
price                           0
transaction_type                0
listing_status                  0
listing_special_features        0
census_state_name               0
census_county_name              0
zip                             0
dtype: int64

<IPython.core.display.Javascript object>

In [87]:
# How many rows are vacant land?
GA_LISTINGS_SALES_V2_df["details"].str.contains("Lots/Land").sum()

5890

<IPython.core.display.Javascript object>

In [89]:
# Dropping these since these sales wont help us
GA_LISTINGS_SALES_V2_df = GA_LISTINGS_SALES_V2_df[~GA_LISTINGS_SALES_V2_df['details'].str.contains('Lots/Land')]


<IPython.core.display.Javascript object>

In [109]:
# About a hundred zip codes have less than 30 listings. We might remove these at a later point


value_counts = GA_LISTINGS_SALES_V2_df["zip"].value_counts()
unique_values = value_counts[value_counts > 30].index
num_unique_values = len(unique_values)
counts_greater_than_30 = value_counts[value_counts > 30]

print(f"The number of original zip codes: {len(zip_list)}")
print(f"The number of zip codes with more than 30 observations: {num_unique_values}")
print(counts_greater_than_30)

The number of original zip codes: 241
The number of zip codes with more than 30 observations: 148
30052.0    245
30318.0    159
30135.0    143
30305.0    123
30132.0    123
30642.0    123
30310.0    119
30157.0    117
30032.0    116
30223.0    116
30253.0    114
30506.0    114
30161.0    113
30281.0    113
30114.0    112
30349.0    111
30143.0    110
30316.0    109
30014.0    109
30004.0    108
30263.0    107
31024.0    102
30315.0    101
30309.0    100
30041.0     98
30040.0     98
30214.0     97
30180.0     95
30252.0     95
30127.0     95
30228.0     94
30016.0     93
30101.0     93
30224.0     92
30024.0     92
30518.0     91
30655.0     91
30043.0     89
30115.0     89
30540.0     89
30680.0     89
30331.0     87
30236.0     87
30120.0     86
30344.0     84
30117.0     83
30188.0     82
30058.0     80
30078.0     80
30240.0     77
30519.0     77
30314.0     76
30319.0     76
30534.0     76
30134.0     76
30327.0     75
30038.0     73
30533.0     73
30012.0     73
30047.0     73
30

<IPython.core.display.Javascript object>

In [110]:
# Saving locally
GA_LISTINGS_SALES_V2_df.to_csv("atlanta_listings.csv", index=False)

<IPython.core.display.Javascript object>

In [111]:
# Saving to AWS
file_name = "atlanta_listings.csv"

# Upload the file to S3
s3.upload_file(file_name, bucket_name, file_name)

<IPython.core.display.Javascript object>